# The first CNN (but first more preparation)
Unfortunately CNNs cannot work with letter sequences directly so we have to think about how to encode the sequences (and our labels) into numerical form, and although CNNs are able to deal with sequences of varying length this would require us to split the training set by length so we are going to pad the sequences to have a uniform length.<br>
First we need our balanced dataset to continue.<br>
<b>Read the balanced dataset from csv into a DataFrame</b>

In [1]:
import pandas as pd

df_balanced = pd.read_csv("df_balanced.csv", index_col=0)
#df_balanced

### ASCII encoding and padding
A simple idea to encode letters is to use their numerical representation according to the ASCII table.<br>
<b>Create a new column in the DataFrame that contains the sequence as a list of decimal numbers according to the ASCII table</b>

In [2]:
df_balanced["seq_ord"] = df_balanced.Seq.map(lambda x: [ord(c) for c in x])
#df_balanced

Now we can use the pad_sequences() function from keras.preprocessing to pad our sequences so they all have the same length (https://keras.io/preprocessing/sequence/).<br>
<b> Use pad_sequences() to create a new column containing the padded ascii encoded sequences.</b>

In [3]:
from keras.preprocessing.sequence import pad_sequences

df_balanced["seq_ord_pad"] = list(pad_sequences(df_balanced.seq_ord.to_numpy()))
#df_balanced

Using TensorFlow backend.


### Another numerical encoding
Since the ASCII representations can be large and also have varying distance between them, we want to reduce the encoding so that it uses only the smallest necessary integers. For this we can use the LabelEncoder from sklearn.preprocessing (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html ).<br>
<b>Use the LabelEncoder to create a column containg the sequences in a minimal numerical encoding</b>

In [4]:
from sklearn.preprocessing import LabelEncoder
import numpy as np # this might be handy

seq_enc = LabelEncoder()
seq_enc.fit(np.hstack(df_balanced.seq_ord_pad.values))
df_balanced["seq_num_pad"] = df_balanced.seq_ord_pad.map(lambda x: seq_enc.transform(x))
print(list(df_balanced.seq_num_pad[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### One hot encoding
A popular method for encdoding categorical features that avoids an implicit order is "one hot encoding". Again scikit-learn offers tools for this (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder ).<br>
<b>Use OneHotEncoder to create column that contains the one hot encoded sequence (use sparse=False)</b>

In [5]:
from sklearn.preprocessing import OneHotEncoder

oh_enc = OneHotEncoder(sparse=False, categories="auto")
oh_enc.fit(np.hstack(df_balanced.seq_num_pad.to_numpy()).reshape(-1, 1))

df_balanced["seq_oh_pad"] = df_balanced.seq_num_pad.map(lambda x: oh_enc.transform(x.reshape(-1,1)))

print(df_balanced.seq_oh_pad[0])

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


### Don't forget the labels
We have to encode the labels as well. Again scikit-learn offers several options but we will just use the LabelEncoder again, since we only have two classes (i.e. 0 or 1).<br>
<b>Create a column with encoded labels using LabelEncoder (save the encoder in a variable so we can use it for inverse transformation later)</b>

In [7]:
from sklearn.preprocessing import LabelEncoder

lbl_enc = LabelEncoder()
df_balanced["lbl_num"] = lbl_enc.fit_transform(df_balanced.Label)
#df_balanced.lbl_num

In [8]:
df_balanced.columns

Index(['Seq', 'Label', 'Length', 'GC_content', 'ATGC_ratio', 'seq_ord',
       'seq_ord_pad', 'seq_num_pad', 'seq_oh_pad', 'lbl_num'],
      dtype='object')

<b>Save all of our hard work to a csv<b>

In [20]:
df_balanced.to_csv("df_balanced_enc.csv")
df_balanced

Seq  \
Arabidopsis_thaliana300001_SnoR1b      GGCGAGGATGAATAATGCTAAATTTCTGACACCTCTTGTATGAGGA...   
Arabidopsis_thaliana300003_SnoR10-1    AGAAATGATGAGAAATCAGATAAATCTTAGGACACCTTCTGACACA...   
Arabidopsis_thaliana300006_SnoR101     GGGATACACTTGATCTCTGAACTTCACAGGTAAGTTCGCTTGTTGA...   
Arabidopsis_thaliana300007_SnoR102     AGAAGTCAATAGACCAGACATTGTGGTAACACTCTCTTTCATGGCA...   
Arabidopsis_thaliana300010_SnoR105     AGGGGATATGATGAATGGTAAAAACTCGCTTATATTGCGAGAAGAG...   
Arabidopsis_thaliana300013_SnoR108     AGTTATAGGGGAAATGAGGAATGGTTATAATCTCGCTTTAATGCGT...   
Arabidopsis_thaliana300015_SnoR11      TGGTTTTGAAGATAGGTTACCTTAATTCTCGGTTTATTAAGTGGAT...   
Arabidopsis_thaliana300020_SnoR12-2    TGAGGGTGATGAAAAAAATCATTCGGATTCCCAATGATTCCCTTAC...   
Arabidopsis_thaliana300021_SnoR12-1b   TGATGGTAAAAGATGACGAAAAAAATCGTTCGGATTCCCTTTGAAT...   
Arabidopsis_thaliana300023_SnoR13-2    AGATCAGGTGATGAGGTAATACTGCAACAATATGATGGAGTAATCT...   
Arabidopsis_thaliana300024_SnoR14-1    CGAGGTGACGATAGAAATTCACAGGTCTGTCAATCCACTGATATAT...   
Arabidopsis_thaliana300025_SnoR14-2    GAAGTGACGATAAAGAATTCACAGTCTGTCAATCCACTGACAACAT...   
Arabidopsis_thaliana300026_SnoR15      GCAGATGATGATTATCATAAAACAAATGGGTAATTTGCGACTGATA...   
Arabidopsis_thaliana300027_SnoR16-1    GCAAATGATGAGTAGAATCTTATCCTACACACAGATGTATCAGTGT...   
Arabidopsis_thaliana300029_SnoR17      CATAGTGATGAATTATATCACAATTGTCACTACCTCTGAGGCATAT...   
Arabidopsis_thaliana300030_SnoR18a     AGATATGATGATGGAATTAGATTTCCCTATTGGTTTGATCGCCGTT...   
Arabidopsis_thaliana300031_SnoR18b     TGAAGTGATGATTGAACTTTGTTTCCCTATTGGTTTGATCGCCGAT...   
Arabidopsis_thaliana300032_SnoR19-1    AACAGTGATGAGTCAGTTTACAGACCTGTAATGATTGCGGTAATGA...   
Arabidopsis_thaliana300034_SnoR20-1    GCCAGTGATGATTAGATTCAATGGTTGCTGAACATTCAATGTTGAA...   
Arabidopsis_thaliana300036_SnoR21b     GGCTGTGATGATTGAGAATGTCATGCACCACTCTGATTTTTTGTGC...   
Arabidopsis_thaliana300041_SnoR22-2    ATGAGCTGATGATGAAAAAAAACAATTATCTTAGCGGATTTCAGTG...   
Arabidopsis_thaliana300042_SnoR23-2    GATGCAATGATGAGATAAAAGCACGAAGAGTGTGAGAAGAGCAGTG...   
Arabidopsis_thaliana300044_SnoR23-1    TGCGATGATGATAAAATAGCACGAAGGTGCGTGAAGGGCAGTGGAC...   
Arabidopsis_thaliana300051_SnoR27      ATTGGATCGATGATTGATTAGTTAAAATACTGCGTTATCTTTCTGC...   
Arabidopsis_thaliana300052_SnoR28-1a   AGGTGATGACAATTTCCCCAGATGATCGAATCTGATTATACTGAGC...   
Arabidopsis_thaliana300054_SnoR28-1c   GGAGGTGATGACAATTTCCCCAAATGATCGAATCAGATTATACTAA...   
Arabidopsis_thaliana300055_SnoR28-2a   GGGAGGTGATGATTATTTCCCCAGATGATCGAAGCTGATTATACGG...   
Arabidopsis_thaliana300056_SnoR28-2b   AGATGGAAGTGAGGATTATTTCTCCAAATGATCGAAGCTGAACAAT...   
Arabidopsis_thaliana300057_SnoR29-2    GAGAAATCGTGGTGATGACTTGGAAATATTCAAGCTCAACAGACCG...   
Arabidopsis_thaliana300058_SnoR29-1    GGCAGTGATGACTCGGAAATTTCAAGCTCAACAGACCGGAATTAGG...   
...                                                                                  ...   
Mus_musculus300900_SNORA62             CCTGCACATTGTTAGAGCTTGGAGTTGAGGTCACACGCTGGTCGAT...   
Saccharomyces_cerevisiae300001_snR85   TACTAGCACATATGTCAAATTCCTAATTAACGTTTTTTATATTCAT...   
Saccharomyces_cerevisiae300002_snR84   ATTGCACAACTTAAGTTTGTCGAGGATCATTTTTTTGAACTGAATC...   
Saccharomyces_cerevisiae300005_snR43   CACTCCTGTTTCTGCCTCATGTCTTTTAAAACGCTTCCACGGGACA...   
Saccharomyces_cerevisiae300006_snR32   AACATCATAATTATATCGCGAATGTACTACTAGTATATGCAGTTTT...   
Saccharomyces_cerevisiae300007_snR81   AGGTTAGGGACTGCAAAAGAAGCGGCGAGGCAGCCCACATCAAGTG...   
Saccharomyces_cerevisiae300009_snR36   TTGCCCTGTGCCTCGCTCGGTTGTTAATTGCCAATATTACGATCTT...   
Saccharomyces_cerevisiae300010_snR49   ATGGGACTTTATCTCCATGACTATGCCATTTCTAGGTAGCCTATGG...   
Saccharomyces_cerevisiae300020_snR82   ATGGCTCTTCAACACATTTCAACATGTTCAAGTAATTTGTGTTAGT...   
Saccharomyces_cerevisiae300021_snR189  AACAAAATAATTTACATTCCAAGGACCGGTATTATTGTAGGGGATT...   
Saccharomyces_cerevisiae300022_snR42   ATAACAATTTGTGATGCTTTAGGGAGCCTATTGTTTGATGGTTTAT...   
Saccharomyces_cerevisiae300023_snR10   AACGCAAATTTAACAGCCATTCGTAACACGTACAGTATCTCGTCGA...   
Saccharomyces_cerevisiae300028_snR80 

## Train/Test
<b>Create a 80/20 train test split, use the encoded labels!</b>

In [10]:
from sklearn.model_selection import train_test_split

rnd_seed=42
xTrain, xTest, yTrain, yTest = train_test_split( df_balanced, df_balanced.lbl_num, test_size=0.2, random_state=rnd_seed )

# Create First convolutional model
Now we are finally ready to create our first model.<br>
The model is going to have the following architecture:
<li>A 1D convolutional layer using an RelU activation with padding="same" (Why?). Use 16 filters of size 9. Use the input_shape parameter </li>
<li>A max pooling layer reducing by a factor of 2</li>
<li>A Flatten layer</li>
<li>A Dense output layer using a sigmoid activation</li>
<li>We will use the Adam optimizer. <i>What is a good choice for the loss function?</i></li><br>

Since we want to use GridSearchCV from sklearn later on we will create a function that returns the compiled model (This is needed for the scikit-learn wraper).<br>
This function should have parameters for the input_shape, because we are going to test our different encoding schemes.
It should also have parameters for the number of filters and their sizes in the first convolutional layer for the grid search. <i>Feel free to insert a print(model.summary()) to get an overview of the model</i><br>We are going to train for 10 epochs using a batch size of 32

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

epochs=10
batch_size=32

def create_first_model(input_shape, c1_filter=16, c1_size=9, verbose=0):
    # create the model
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(c1_filter, c1_size, activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Flatten())    
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    if verbose:
        print(model.summary())
    
    return model


## Test different encodings
Now we will test the different encoding techniques with the first model. For each test the KerasClassifier wrapper will be used to create the model (https://keras.io/scikit-learn-api/ )
<br>The model should then be trained with the training data and a classification report for the predictions on the test data should be generated. You can use th inverse_transform function of the LabelEncoder to have readable labels in the classification report.<br>Use the verbose option while fitting to monitor training progress.

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report

### First model with ASCII encoded sequences
<b> Create a KerasClassifier, train the model using the ASCII encoded training data and evaluate the model (numpy.reshape might come in handy)</b>

In [25]:
model = KerasClassifier(build_fn=create_first_model, input_shape=(1004,1), verbose=1)
model.fit(np.array(xTrain.seq_ord_pad.to_list()).reshape(-1,1004,1), yTrain.to_numpy(), epochs=epochs, batch_size=batch_size, verbose=0)
yPred = model.predict(np.array(xTest.seq_ord_pad.to_list()).reshape(-1,1004,1))
print(classification_report( lbl_enc.inverse_transform(yTest.to_list()), lbl_enc.inverse_transform(yPred.ravel())))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 1004, 16)          160       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 502, 16)           0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 8032)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 8033      
Total params: 8,193
Trainable params: 8,193
Non-trainable params: 0
_________________________________________________________________
None
197/197 [==============================] - 0s 468us/sample
              precision    recall  f1-score   support

      CD-box       0.88      0.92      0.90        99
    HACA-box       0.91      0.87      0.89        98

    accuracy                           0.89 

### First model with numerically encoded sequences
<b> Create a KerasClassifier, train the model using the numerically encoded training data and evaluate the model (numpy.reshape might come in handy)</b>

In [18]:
model = KerasClassifier(build_fn=create_first_model, input_shape=(1004,1), verbose=0)
model.fit(np.array(xTrain.seq_num_pad.to_list()).reshape(787,1004,1), yTrain.to_numpy(), epochs=epochs, batch_size=batch_size, verbose=0) # start training
yPred = model.predict(np.array(xTest.seq_num_pad.to_list()).reshape(-1,1004,1))
print(classification_report( yTest.to_list(), yPred ))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91        99
           1       0.93      0.87      0.90        98

    accuracy                           0.90       197
   macro avg       0.91      0.90      0.90       197
weighted avg       0.91      0.90      0.90       197



## First model with one hot encoded sequences
<b> Create a KerasClassifier, train the model using the one hot encoded training data and evaluate the model (numpy.reshape might come in handy)</b>

In [19]:
model = KerasClassifier(build_fn=create_first_model, input_shape=(1004,5), verbose=1)
model.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(787,1004,5), yTrain.to_numpy(), epochs=epochs, batch_size=batch_size, verbose=1) # start training
yPred = model.predict(np.array(xTest.seq_oh_pad.to_list()).reshape(-1,1004,5))
print(classification_report( yTest.to_list(), yPred ))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 1004, 16)          736       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 502, 16)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 8032)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 8033      
Total params: 8,769
Trainable params: 8,769
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
787/787 [==============================] - 0s 432us/sample - loss: 0.5784 - acc: 0.6963
Epoch 2/10
787/787 [==============================] - 0s 308us/sample - loss: 0.3546 - acc: 0.8717
Epoch 3/10
787/787 [==============================] - 0s 276us/sample

# Results
Which encoding performed best?<br>
Re-run the cells. Are the results stable?<br>
Are all CNNs used the same?

# GridSearch for first convolutional layer
From now on we will only use the one hot encoded sequences as they performed best with this model so far.<br>
Let's examine the influence of the number of kernels and their sizes using GridSearchCV from sklearn.model_selection (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV )<br>
Define 

In [108]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_first_model, input_shape=(1004,5), verbose=1)

# define the grid search parameters
c1_filters = [4,16,32,64,96]
c1_sizes = [3,9,19,39,79]

param_grid = dict(c1_filter=c1_filters,
                  c1_size=c1_sizes
)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_result = grid.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(-1,1004,5), yTrain.to_numpy())

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))



Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/wehrmann/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  1.4min finished
/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_36 (Conv1D)           (None, 1004, 96)          38016     
_________________________________________________________________
average_pooling1d_36 (Averag (None, 502, 96)           0         
_________________________________________________________________
flatten_35 (Flatten)         (None, 48192)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 48193     
Total params: 86,209
Trainable params: 86,209
Non-trainable params: 0
_________________________________________________________________
None
787/787 [==============================] - 2s 3ms/sample - loss: 0.4631 - acc: 0.7916
Best: 0.891995 using {'c1_filter': 96, 'c1_size': 79}


In [114]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param.values()))

0.740788 (0.102275) with: dict_values([4, 3])
0.794155 (0.082565) with: dict_values([4, 9])
0.571792 (0.068328) with: dict_values([4, 19])
0.714104 (0.094760) with: dict_values([4, 39])
0.866582 (0.023269) with: dict_values([4, 79])
0.763659 (0.167053) with: dict_values([16, 3])
0.832274 (0.041975) with: dict_values([16, 9])
0.878018 (0.021331) with: dict_values([16, 19])
0.858958 (0.029449) with: dict_values([16, 39])
0.881830 (0.030432) with: dict_values([16, 79])
0.804320 (0.082626) with: dict_values([32, 3])
0.865311 (0.031533) with: dict_values([32, 9])
0.883100 (0.024932) with: dict_values([32, 19])
0.843710 (0.045223) with: dict_values([32, 39])
0.888183 (0.035183) with: dict_values([32, 79])
0.688691 (0.160551) with: dict_values([64, 3])
0.848793 (0.049138) with: dict_values([64, 9])
0.874206 (0.020192) with: dict_values([64, 19])
0.874206 (0.019207) with: dict_values([64, 39])
0.890724 (0.022737) with: dict_values([64, 79])
0.837357 (0.011966) with: dict_values([96, 3])
0.8437

# visualize gridsearch

In [117]:
import matplotlib.pyplot as plt

mean_scores = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
filters = [d["c1_filter"] for d in params]
sizes = [d["c1_size"] for d in params]

In [127]:
df_gridsearch = pd.DataFrame(zip(scores, filters, sizes), columns=["mean_test_score","filters","sizes"])
#df_gridsearch_piv = pd.pivot_table(df_gridsearch, values=df_gridsearch["mean_test_score"])
df_gridsearch.pivot_table(index="filters", columns="sizes")
#df_gridsearch_piv

mean_test_score                                        
sizes                3         9         19        39        79
filters                                                        
4              0.740788  0.794155  0.571792  0.714104  0.866582
16             0.763659  0.832274  0.878018  0.858958  0.881830
32             0.804320  0.865311  0.883100  0.843710  0.888183
64             0.688691  0.848793  0.874206  0.874206  0.890724
96             0.837357  0.843710  0.875476  0.856417  0.891995

In [130]:
import seaborn as sns

sns.heatmap(df_gridsearch.pivot_table(index="filters", columns="sizes"), annot=True, cmap="Blues"); #annot=True to annotate cells

In [ ]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_first_model, input_shape=(1004,5), verbose=1)

# define the grid search parameters
c1_filters = [4,16,32,64,96]
c1_sizes = [3,9,19,39,79]

param_grid = dict(c1_filter=c1_filters,
                  c1_size=c1_sizes
)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_result = grid.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(-1,1004,5), yTrain.to_numpy())

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [131]:
model = KerasClassifier(build_fn=create_first_model, input_shape=(1004,5), c1_filter=32, c1_size=19, verbose=1)
model.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(787,1004,5), yTrain.to_numpy(), epochs=epochs, batch_size=batch_size, verbose=1) # start training
yPred = model.predict(np.array(xTest.seq_oh_pad.to_list()).reshape(-1,1004,5))
print(classification_report( yTest.to_list(), yPred ))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_37 (Conv1D)           (None, 1004, 32)          3072      
_________________________________________________________________
average_pooling1d_37 (Averag (None, 502, 32)           0         
_________________________________________________________________
flatten_36 (Flatten)         (None, 16064)             0         
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 16065     
Total params: 19,137
Trainable params: 19,137
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.5152 - acc: 0.7738
Epoch 2/10
787/787 [==============================] - 0s 581us/sample - loss: 0.2662 - acc: 0.9022
Epoch 3/10
787/787 [==============================] - 0s 566us/sample

In [133]:
model = grid_result.best_estimator_
yPred = model.predict(np.array(xTest.seq_oh_pad.to_list()).reshape(-1,1004,5))
print(classification_report( yTest.to_list(), yPred ))

197/197 [==============================] - 1s 3ms/sample
              precision    recall  f1-score   support

           0       0.84      0.98      0.91        99
           1       0.98      0.82      0.89        98

    accuracy                           0.90       197
   macro avg       0.91      0.90      0.90       197
weighted avg       0.91      0.90      0.90       197



# Create second model

In [438]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_second_model(input_shape, c1_filter=16, c1_size=9, c2_filter=8, c2_size=48):
    # create the model
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(c1_filter, c1_size, activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.Conv1D(filters=c2_filter, kernel_size=c2_size, activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.AveragePooling1D(2))
    model.add(keras.layers.Flatten())    
    #model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


In [439]:
model = KerasClassifier(build_fn=create_second_model, input_shape=(1004,6))
model.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(787,1004,6), yTrain.to_numpy(), epochs=epochs, batch_size=batch_size, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_155 (Conv1D)          (None, 1004, 16)          880       
_________________________________________________________________
conv1d_156 (Conv1D)          (None, 1004, 8)           6152      
_________________________________________________________________
average_pooling1d_13 (Averag (None, 502, 8)            0         
_________________________________________________________________
flatten_95 (Flatten)         (None, 4016)              0         
_________________________________________________________________
dense_192 (Dense)            (None, 1)                 4017      
Total params: 11,049
Trainable params: 11,049
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
787/787 [==============================] - 3s 4ms/sample - loss: 0.5961 - acc: 0.7294
Epoch 2/5
787/787 [==================

In [ ]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_first_model, verbose=1)
# define the grid search parameters
input_shapes = [(1004,6)]
batch_sizes = [32]
num_epochs = [5,10]
c1_filters = np.linspace(2,100,10, dtype=int)
c1_sizes = [9,19,39,79]#np.linspace(3,127,10, dtype=int)

param_grid = dict(#batch_size=batch_size,
                  #epochs=epochs,
                  input_shape=input_shapes,
                  c1_filter=c1_filters,
                  c1_size=c1_sizes
)


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=1)

grid_result = grid.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(787,1004,6), yTrain.to_numpy())

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']

#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

In [440]:
yPred = model.predict(np.array(xTest.seq_oh_pad.to_list()).reshape(-1,1004,6))
print(classification_report( yTest.to_list(), yPred ))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91        99
           1       0.92      0.89      0.90        98

    accuracy                           0.90       197
   macro avg       0.90      0.90      0.90       197
weighted avg       0.90      0.90      0.90       197



In [442]:
model = KerasClassifier(build_fn=create_second_model, verbose=1)
# define the grid search parameters
input_shapes = [(1004,6)]
batch_sizes = [32]
num_epochs = [5,10]
c2_filters = np.linspace(2,100,10, dtype=int)
c2_sizes = [9,19,39,79,127,201]#np.linspace(3,127,10, dtype=int)

param_grid = dict(#batch_size=batch_size,
                  epochs=num_epochs,
                  input_shape=input_shapes,
                  c2_filter=c2_filters,
                  c2_size=c2_sizes
)


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=1)

grid_result = grid.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(787,1004,6), yTrain.to_numpy())

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']

#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 2 folds for each of 120 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 30.2min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 45.1min finished


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_157 (Conv1D)          (None, 1004, 16)          880       
_________________________________________________________________
conv1d_158 (Conv1D)          (None, 1004, 78)          48750     
_________________________________________________________________
average_pooling1d_14 (Averag (None, 502, 78)           0         
_________________________________________________________________
flatten_96 (Flatten)         (None, 39156)             0         
_________________________________________________________________
dense_193 (Dense)            (None, 1)                 39157     
Total params: 88,787
Trainable params: 88,787
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
787/787 [==============================] - 4s 5ms/sample - loss: 0.4337 - acc: 0.7992
Epoch 2/10
787/787 [================

In [443]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.907243 (0.011318) with: {'c2_filter': 2, 'c2_size': 9, 'epochs': 5, 'input_shape': (1004, 6)}
0.908513 (0.015132) with: {'c2_filter': 2, 'c2_size': 9, 'epochs': 10, 'input_shape': (1004, 6)}
0.894536 (0.016384) with: {'c2_filter': 2, 'c2_size': 19, 'epochs': 5, 'input_shape': (1004, 6)}
0.909784 (0.011321) with: {'c2_filter': 2, 'c2_size': 19, 'epochs': 10, 'input_shape': (1004, 6)}
0.905972 (0.017670) with: {'c2_filter': 2, 'c2_size': 39, 'epochs': 5, 'input_shape': (1004, 6)}
0.904701 (0.021480) with: {'c2_filter': 2, 'c2_size': 39, 'epochs': 10, 'input_shape': (1004, 6)}
0.909784 (0.008780) with: {'c2_filter': 2, 'c2_size': 79, 'epochs': 5, 'input_shape': (1004, 6)}
0.905972 (0.012587) with: {'c2_filter': 2, 'c2_size': 79, 'epochs': 10, 'input_shape': (1004, 6)}
0.889454 (0.008754) with: {'c2_filter': 2, 'c2_size': 127, 'epochs': 5, 'input_shape': (1004, 6)}
0.905972 (0.012587) with: {'c2_filter': 2, 'c2_size': 127, 'epochs': 10, 'input_shape': (1004, 6)}
0.908513 (0.002425) with:

In [501]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_vgg_model(input_shape, conv2_kernels=16, conv2_width=32):
    # create the model
    model = keras.Sequential()
    #model.add(keras.layers.Embedding(6, 2, input_length=1004 ) )
    model.add(keras.layers.Conv1D(64, 7, activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.Conv1D(64, 13, activation='relu', padding='same'))
    model.add(keras.layers.Conv1D(64, 27, activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(16, 13, activation='relu'))
    #model.add(keras.layers.Conv1D(16, 13, activation='relu'))
    #model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Flatten())
    #model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


In [504]:
model = KerasClassifier(build_fn=create_vgg_model, input_shape=(1004,6))
model.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(787,1004,6), yTrain.to_numpy(), epochs=20, batch_size=batch_size, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_212 (Conv1D)          (None, 1004, 64)          2752      
_________________________________________________________________
conv1d_213 (Conv1D)          (None, 1004, 64)          53312     
_________________________________________________________________
conv1d_214 (Conv1D)          (None, 1004, 64)          110656    
_________________________________________________________________
max_pooling1d_144 (MaxPoolin (None, 502, 64)           0         
_________________________________________________________________
flatten_117 (Flatten)        (None, 32128)             0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32128)             0         
_________________________________________________________________
dense_222 (Dense)            (None, 1)                 32129     
Total para

In [503]:
yPred = model.predict(np.array(xTest.seq_oh_pad.to_list()).reshape(-1,1004,6))
print(classification_report( yTest.to_list(), yPred ))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96        99
           1       0.96      0.96      0.96        98

    accuracy                           0.96       197
   macro avg       0.96      0.96      0.96       197
weighted avg       0.96      0.96      0.96       197



In [170]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report

## First model with ASCII encoded sequences

In [190]:
model = KerasClassifier(build_fn=create_second_model, input_shape=(1004,1), verbose=100)
model.fit(np.array(xTrain.seq_ord_pad.to_list()).reshape(787,1004,1), yTrain.to_numpy(), epochs=10, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_60 (Conv1D)           (None, 1004, 16)          1040      
_________________________________________________________________
max_pooling1d_58 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_59 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_57 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_58 (Dense)             (None, 1)                 21        
Total para

In [191]:
yPred = model.predict(np.array(xTest.seq_ord_pad.to_list()).reshape(-1,1004,1))
print(classification_report( yTest.to_list(), yPred ))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88        99
           1       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



## First model with numerically encoded sequences

In [192]:
model = KerasClassifier(build_fn=create_second_model, input_shape=(1004,1), verbose=100)
model.fit(np.array(xTrain.seq_num_pad.to_list()).reshape(787,1004,1), yTrain.to_numpy(), epochs=10, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_62 (Conv1D)           (None, 1004, 16)          1040      
_________________________________________________________________
max_pooling1d_60 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_61 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_59 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 21        
Total para

In [193]:
yPred = model.predict(np.array(xTest.seq_num_pad.to_list()).reshape(-1,1004,1))
print(classification_report( yTest.to_list(), yPred ))

              precision    recall  f1-score   support

           0       0.90      0.93      0.92        99
           1       0.93      0.90      0.91        98

    accuracy                           0.91       197
   macro avg       0.91      0.91      0.91       197
weighted avg       0.91      0.91      0.91       197



## Second model with one hot encoded sequences

In [194]:
model = KerasClassifier(build_fn=create_second_model, input_shape=(1004,6), verbose=100)
model.fit(np.array(xTrain.seq_oh_pad.to_list()).reshape(787,1004,6), yTrain.to_numpy(), epochs=10, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_64 (Conv1D)           (None, 1004, 16)          6160      
_________________________________________________________________
max_pooling1d_62 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_63 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 21        
Total para

In [195]:
yPred = model.predict(np.array(xTest.seq_oh_pad.to_list()).reshape(-1,1004,6))

print(classification_report( yTest.to_list(), yPred ))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        99
           1       0.92      0.92      0.92        98

    accuracy                           0.92       197
   macro avg       0.92      0.92      0.92       197
weighted avg       0.92      0.92      0.92       197



In [159]:
len(yTrain.to_numpy())

787

In [145]:
np.array(xTrain.seq_num_pad.to_list()).reshape(787,1004,1).shape

(787, 1004, 1)

In [161]:
model = KerasClassifier(build_fn=create_first_model, input_shape=(1004,1), verbose=100)
model.fit(np.array(xTrain.seq_num_pad.to_list()).reshape(787,1004,1), yTrain.to_numpy(), epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_42 (Conv1D)           (None, 1004, 16)          112       
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7776)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 21        
Total para

In [142]:
xTrain.seq_num_pad.to_numpy().shape

(787,)

In [ ]:
print(xTrain.seq_oh_pad.to_numpy().shape)
xTrain.seq_oh_pad.to_numpy()[0].shape

In [129]:


model = KerasClassifier(build_fn=create_model_num, verbose=1)

model.fit(xTrain.seq_num_pad.to_numpy(), yTrain, epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_26 (Conv1D)           (None, 1004, 16)          112       
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
dense_23 (Dense)             (None, 243, 20)           660       
_________________________________________________________________
dense_24 (Dense)             (None, 243, 1)            21        
Total params: 9,017
Trainable params: 9,017
Non-trainable params: 0
_________________________________________________________________
None


ValueError: Error when checking input: expected conv1d_26_input to have 3 dimensions, but got array with shape (787, 1)

## test our first model with numerically encoded sequences

In [77]:
yPred = model.predict(xTest_num.reshape(-1,1004,1))
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( yTest.Class.to_list(), encoder.inverse_transform(yPred) ))

197/197 [==============================] - 0s 1ms/sample
              precision    recall  f1-score   support

      CD-box       0.87      0.90      0.89        99
    HACA-box       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



# ads "############"

In [75]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_num(dropout=0.5):
    # create the model
    model = keras.Sequential()
    #model.add(keras.layers.Embedding(6, 3, input_length=1004 ) )
    model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same', input_shape=(1004,1)))
    #model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(32, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(64, 32, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(128, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    #model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


## train our first model with numerically encoded sequences

In [76]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_num, verbose=1)

#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

model.fit(xTrain_num.reshape(787,1004,1), yTrain_enc, epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_40 (Conv1D)           (None, 1004, 16)          112       
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 21        
Total para

## test our first model with numerically encoded sequences

In [77]:
yPred = model.predict(xTest_num.reshape(-1,1004,1))
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( yTest.Class.to_list(), encoder.inverse_transform(yPred) ))

197/197 [==============================] - 0s 1ms/sample
              precision    recall  f1-score   support

      CD-box       0.87      0.90      0.89        99
    HACA-box       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



In [82]:
xTrain.seq_num.to_numpy().reshape(-1,1,1)

array([[[array([0, 4, 2, 2, 1, 4, 1, 4, 4, 1, 0, 0, 1, 0, 1, 0, 4, 4, 4, 1, 0, 0,
       1, 0, 4, 2, 4, 4, 1, 0, 0, 2, 4, 0, 0, 4, 4, 4, 2, 4, 2, 4, 4, 0,
       2, 4, 2, 2, 0, 4, 2, 0, 1, 1, 0, 4, 4, 4, 0, 2, 2, 2, 2, 1, 4, 2,
       1, 4, 2, 2, 1, 1, 4, 2, 2, 4, 4, 0, 1, 1, 2, 2, 2, 0, 2, 4, 4, 4,
       4, 4, 1, 4, 4, 2, 2, 0, 4, 1, 1, 0, 0, 2, 1, 4, 0, 2, 1, 4, 4, 4,
       4, 1, 1, 2, 4, 1, 4, 2, 0, 4, 4, 0, 4, 1, 1, 4, 4, 0, 0, 2, 1, 4,
       4, 1, 0, 1, 0, 0, 0, 4, 4, 0, 1, 0, 0, 4, 4, 4, 4, 4, 1, 1, 1, 0,
       1, 2, 1, 0, 4, 4, 0, 0, 2, 0, 0, 0, 4, 0, 0, 2, 1, 4, 1, 0, 0, 2,
       0, 4, 2, 1, 1, 4, 0, 0, 0, 0, 4, 0, 0, 2, 4, 4, 1, 4, 0, 4, 1, 1,
       1, 2, 1, 1, 4, 4, 4, 4, 4, 4, 1, 2, 1, 4, 0, 0, 1, 0, 0, 4, 2, 0,
       1, 4, 2, 0, 2, 4, 0, 4, 4, 1, 1, 1, 0, 1, 0, 2, 4, 1, 4, 0, 4, 0,
       2, 4, 4, 4, 2, 0, 4, 0, 2, 4, 0, 2, 0, 4, 2, 2, 2, 1, 2, 2, 0, 0,
       0, 4, 4, 4])]],

       [[array([0, 4, 0, 1, 1, 4, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 4, 1, 0, 1, 4,
       4,

In [86]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_num, verbose=1)

#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

model.fit(xTrain.seq_num.to_numpy().reshape(-1,1), yTrain, epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, None, 16)          112       
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, None, 16)          0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, None, 32)          8224      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, None, 32)          0         
_________________________________________________________________
dense_9 (Dense)              (None, None, 20)          660       
_________________________________________________________________
dense_10 (Dense)             (None, None, 1)           21        
Total params: 9,017
Trainable params: 9,017
Non-trainable params: 0
_________________________________________________________________
None


ValueError: Error when checking input: expected conv1d_12_input to have 3 dimensions, but got array with shape (787, 1)

## test our first model with numerically encoded sequences

In [77]:
yPred = model.predict(xTest_num.reshape(-1,1004,1))
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( yTest.Class.to_list(), encoder.inverse_transform(yPred) ))

197/197 [==============================] - 0s 1ms/sample
              precision    recall  f1-score   support

      CD-box       0.87      0.90      0.89        99
    HACA-box       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



In [1]:


xTrain = pd.read_csv("xTrain.csv", index_col=0)
xTest = pd.read_csv("xTest.csv", index_col=0)
yTrain = pd.read_csv("yTrain.csv", index_col=0)
yTest = pd.read_csv("yTest.csv", index_col=0)

In [7]:
set.union(*xTrain.Seq.map(set).to_list())

{'A', 'C', 'G', 'N', 'T'}

In [8]:
set.union(*xTest.Seq.map(set).to_list())

{'A', 'C', 'G', 'T'}

In [14]:
xTrain[xTrain.Seq.str.contains("N")].Seq.to_list()

['TTTCTGAGATCTGCTTTTAGTGAAGTGGATCAATGATGAAACTAGCCAAATCTGAGCATCAGAAGTCTTTCCAGTCTACCTGATGCATGATCTCTACAGTTCTGAGAAGCAAAACTATAAAACAATGTAAAACAATAAGGGCATATGTCTGGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTACGCACATGTGTTTATAAAGATAACAGCTGTAGGAATGAATGAGATTGAGGGTGGGGGGGTGCGTATGTATGTCTATGAAAGCCTAATCNTTTCTGGGCAATGATGAAAAGGTTTTACTACTGATCTTTGTAACTATGATGGTTTCTACACTTGACCTGAGCTCA']

In [5]:
from keras.preprocessing.sequence import pad_sequences

xTrain_pad = pad_sequences(xTrain.Seq.map(lambda x: [ord(c) for c in x]), maxlen=max_len)
xTest_pad = pad_sequences(xTest.Seq.map(lambda x: [ord(c) for c in x]), maxlen=max_len)

In [6]:
np.concatenate((xTrain_pad, xTest_pad))

array([[ 0,  0,  0, ..., 84, 84, 84],
       [ 0,  0,  0, ..., 84, 67, 67],
       [ 0,  0,  0, ..., 84, 67, 65],
       ...,
       [ 0,  0,  0, ..., 84, 67, 65],
       [ 0,  0,  0, ..., 65, 65, 65],
       [ 0,  0,  0, ..., 71, 84, 65]], dtype=int32)

In [39]:
from sklearn.preprocessing import OneHotEncoder

ohenc = OneHotEncoder(sparse=False, categories="auto")
ohenc.fit(np.concatenate((xTrain_pad, xTest_pad)))

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=False)

In [43]:
ohenc.categories_

[array([ 0, 84], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 65], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 71], dtype=int32),
 array([ 0, 84], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([ 0, 67], dtype=int32),
 array([

In [40]:
print(ohenc.transform(xTrain_pad))

[[1. 0. 1. ... 0. 0. 1.]
 [1. 0. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 0. 1.]
 [1. 0. 1. ... 1. 0. 0.]]


In [41]:
ohenc.transform(xTrain_pad).shape

(787, 3365)

In [44]:
ohenc.transform(xTest_pad).shape

(197, 3365)

In [42]:
xTrain_pad.shape

(787, 1004)

In [56]:
xTrain_oh = ohenc.transform(xTrain_pad)
xTrain_oh.shape

(787, 3365)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


In [52]:
from sklearn.preprocessing import LabelEncoder

# convert labels to numbers 
encoder = LabelEncoder()
encoder.fit(yTrain.Class.to_list())
yTrain_enc = encoder.transform(yTrain.Class.to_list())

print(encoder.classes_)
print(encoder.transform(encoder.classes_))

['CD-box' 'HACA-box']
[0 1]


In [119]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_embedding(emb_in=6, emb_out=3, conv1_ks=16, conv1_w=3, conv2_ks=32, conv2_w=9, dense=13):
    # create the model
    model = keras.Sequential()
    model.add(keras.layers.Embedding(emb_in, emb_out, input_length=3365 , input_shape=(3365, ) ) )
    model.add(keras.layers.Conv1D(conv1_ks, conv1_w, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(conv2_ks, conv2_w, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(32, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    #model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(dense, activation='relu'))
    #model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(2, activation=tf.nn.softmax))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [115]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_embedding, verbose=1)
#model.summary() #print model summary
model.fit(xTrain_oh, yTrain_enc, epochs=10, batch_size=32, verbose=1) # start training

Epoch 1/10
787/787 [==============================] - 1s 2ms/sample - loss: 0.6805 - acc: 0.7230
Epoch 2/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.5383 - acc: 0.8653
Epoch 3/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.3081 - acc: 0.8920
Epoch 4/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2552 - acc: 0.9111
Epoch 5/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2553 - acc: 0.9072
Epoch 6/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2297 - acc: 0.9187
Epoch 7/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2162 - acc: 0.9301
Epoch 8/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.2057 - acc: 0.9187
Epoch 9/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.1945 - acc: 0.9365
Epoch 10/10
787/787 [==============================] - 1s 1ms/sample - loss: 0.1759 - acc: 0.9441


In [123]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model_embedding, verbose=1)
# define the grid search parameters
batch_size = [32]
epochs = [5]
emb_in=[3,6,9] #6
emb_out=[2,3,4] #3
conv1_ks=[8,16,32] #16
conv1_w=[3,5]
conv2_ks=[8,16,32]
conv2_w=[7,9]
dense=[5,10,20,40]

param_grid = dict(batch_size=batch_size,
                 epochs=epochs,
                 emb_in=emb_in,
                 emb_out=emb_out,
                 conv1_ks=conv1_ks,
                 conv1_w=conv1_w,
                 conv2_ks=conv2_ks,
                 conv2_w=conv2_w,
                 dense=dense)



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=1)

grid_result = grid.fit(xTrain_oh, yTrain_enc)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 2 folds for each of 1296 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/wehrmann/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 25.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 51.8min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 82.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 129.1min
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed: 141.0min finished
/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The defau

Epoch 1/5
787/787 [==============================] - 2s 3ms/sample - loss: 0.6987 - acc: 0.4828
Epoch 2/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.6830 - acc: 0.7001
Epoch 3/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.5211 - acc: 0.8463
Epoch 4/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2746 - acc: 0.9098
Epoch 5/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2315 - acc: 0.9250
Best: 0.921220 using {'batch_size': 32, 'conv1_ks': 16, 'conv1_w': 5, 'conv2_ks': 32, 'conv2_w': 9, 'dense': 40, 'emb_in': 6, 'emb_out': 3, 'epochs': 5}
0.739517 (0.136290) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 2, 'epochs': 5}
0.677255 (0.211338) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 3, 'epochs': 5}
0.673443 (0.207531) with: {'batch_size': 32, 'conv1_ks': 

In [124]:
grid_result.best_params_

{'batch_size': 32,
 'conv1_ks': 16,
 'conv1_w': 5,
 'conv2_ks': 32,
 'conv2_w': 9,
 'dense': 40,
 'emb_in': 6,
 'emb_out': 3,
 'epochs': 5}

In [125]:
model = grid_result.best_estimator_

In [126]:
yPred = model.predict(ohenc.transform(xTest_pad))
yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred_class))

197/197 [==============================] - 0s 2ms/sample
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        99
           1       0.00      0.00      0.00        98

    accuracy                           0.50       197
   macro avg       0.25      0.50      0.33       197
weighted avg       0.25      0.50      0.34       197



/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
len(xTrain_oh[0])

3365

In [91]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

# create the model
model = keras.Sequential()
#model.add(keras.layers.Embedding(5, 3, input_length=3365 , input_shape=(3365, ) ) )
#model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same'))
model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
model.add(keras.layers.MaxPooling1D(2))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.MaxPooling1D(2))
#model.add(keras.layers.Conv1D(32, 16, activation='relu'))
#model.add(keras.layers.Conv1D(32, 32, activation='relu'))
#model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(16, activation='relu'))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation=tf.nn.softmax))

optAdam = keras.optimizers.Adam()
model.compile(optimizer=optAdam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#print model summary
model.summary()

# Fit the model
model.fit(xTrain_oh, yTrain_enc, epochs=10, batch_size=32, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_58 (Conv1D)           (None, 3365, 8)           32        
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 1682, 8)           0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 1676, 16)          912       
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 838, 16)           0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 13408)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 32)                429088    
_________________________________________________________________
dense_37 (Dense)             (None, 2)                 66        
Total para

ValueError: Error when checking input: expected conv1d_58_input to have 3 dimensions, but got array with shape (787, 3365)

In [97]:
yPred = model.predict(ohenc.transform(xTest_pad)) 
yPred

array([[0.03793205, 0.96206796],
       [0.6027639 , 0.39723617],
       [0.04488635, 0.9551136 ],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.05762911, 0.9423709 ],
       [0.03640307, 0.963597  ],
       [0.6027639 , 0.39723617],
       [0.02395522, 0.9760448 ],
       [0.3024247 , 0.6975753 ],
       [0.36231595, 0.63768405],
       [0.09731042, 0.90268964],
       [0.05010566, 0.9498943 ],
       [0.02146999, 0.97853   ],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.49592173, 0.50407827],
       [0.06055814, 0.93944186],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.01738231, 0.9826177 ],
       [0.6027639 , 0.39723617],
       [0.05334512, 0.94665486],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.6027639 , 0.39723617],
       [0.

In [94]:
yPred_class = np.array([np.argmax(row) for row in yPred])
yPred_class

array([1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1])

In [95]:
from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred_class))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90        99
           1       0.91      0.88      0.90        98

    accuracy                           0.90       197
   macro avg       0.90      0.90      0.90       197
weighted avg       0.90      0.90      0.90       197



# Playground best convs and stuff

## create numerically encoded version of train/test data

In [60]:
from sklearn.preprocessing import LabelEncoder

base_encoder = LabelEncoder()
base_encoder.fit(np.concatenate((xTrain_pad, xTest_pad), axis=None))

xTrain_num = np.array([base_encoder.transform(seq) for seq in xTrain_pad])
xTest_num = np.array([base_encoder.transform(seq) for seq in xTest_pad])

print(xTrain_num.shape)
xTrain_num.reshape(787,1004,1)

(787, 1004)


array([[[0],
        [0],
        [0],
        ...,
        [5],
        [5],
        [5]],

       [[0],
        [0],
        [0],
        ...,
        [5],
        [2],
        [2]],

       [[0],
        [0],
        [0],
        ...,
        [5],
        [2],
        [1]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [5],
        [3],
        [3]],

       [[0],
        [0],
        [0],
        ...,
        [3],
        [1],
        [5]],

       [[0],
        [0],
        [0],
        ...,
        [2],
        [1],
        [2]]])

In [61]:
xTrain_num

array([[0, 0, 0, ..., 5, 5, 5],
       [0, 0, 0, ..., 5, 2, 2],
       [0, 0, 0, ..., 5, 2, 1],
       ...,
       [0, 0, 0, ..., 5, 3, 3],
       [0, 0, 0, ..., 3, 1, 5],
       [0, 0, 0, ..., 2, 1, 2]])

## create numerically encoded version of train/test labels

In [17]:
from sklearn.preprocessing import LabelEncoder

# convert labels to numbers 
encoder = LabelEncoder()
encoder.fit(yTrain.Class.to_list())

yTrain_enc = encoder.transform(yTrain.Class.to_list())

print(encoder.classes_)
print(encoder.transform(encoder.classes_))

['CD-box' 'HACA-box']
[0 1]


## create our first model

In [75]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_num(dropout=0.5):
    # create the model
    model = keras.Sequential()
    #model.add(keras.layers.Embedding(6, 3, input_length=1004 ) )
    model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same', input_shape=(1004,1)))
    #model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(32, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(64, 32, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(128, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    #model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


## train our first model with numerically encoded sequences

In [76]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_num, verbose=1)

#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

model.fit(xTrain_num.reshape(787,1004,1), yTrain_enc, epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_40 (Conv1D)           (None, 1004, 16)          112       
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 21        
Total para

## test our first model with numerically encoded sequences

In [77]:
yPred = model.predict(xTest_num.reshape(-1,1004,1))
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( yTest.Class.to_list(), encoder.inverse_transform(yPred) ))

197/197 [==============================] - 0s 1ms/sample
              precision    recall  f1-score   support

      CD-box       0.87      0.90      0.89        99
    HACA-box       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



# One hot encoded sequences

In [124]:
from sklearn.preprocessing import OneHotEncoder

print(xTrain_num.shape)
print(xTrain_num[0].reshape(-1, 1))
xTrain_num_resh =  xTrain_num.T.reshape(1004,787)
#print(xTrain_num_resh.T)
onehotencoder = OneHotEncoder(categories='auto', dtype=int, sparse=False)
print( np.concatenate( (xTrain_num.reshape(-1, 1), xTest_num.reshape(-1, 1)), axis=0 ) )
onehotencoder.fit( np.concatenate( (xTrain_num.reshape(-1, 1), xTest_num.reshape(-1, 1)), axis=0 ) )

xTrain_ohe = np.array([onehotencoder.transform(seq.reshape(-1, 1)) for seq in xTrain_num])
xTest_ohe = np.array([onehotencoder.transform(seq.reshape(-1, 1)) for seq in xTest_num])

print(np.array(xTrain_ohe).shape)
#xTrain_ohe = np.array(xTrain_ohe)

(787, 1004)
[[0]
 [0]
 [0]
 ...
 [5]
 [5]
 [5]]
[[0]
 [0]
 [0]
 ...
 [3]
 [5]
 [1]]
(787, 1004, 6)


## create our second model (for one hot encoding)

In [122]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_ohe(dropout=0.5):
    # create the model
    model = keras.Sequential()
    #model.add(keras.layers.Embedding(6, 3, input_length=1004 ) )
    model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same', input_shape=(1004,6)))
    #model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(32, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(64, 32, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(128, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    #model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dense(20, activation='relu'))
    #model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


## train second model with numerically encoded sequences

In [127]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_ohe, verbose=1)

#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

model.fit(xTrain_ohe, yTrain_enc, epochs=5, batch_size=32, verbose=1) # start training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_46 (Conv1D)           (None, 1004, 16)          592       
_________________________________________________________________
max_pooling1d_37 (MaxPooling (None, 502, 16)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 487, 32)           8224      
_________________________________________________________________
max_pooling1d_38 (MaxPooling (None, 243, 32)           0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 7776)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 20)                155540    
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 21        
Total para

## test our first model with numerically encoded sequences

In [128]:
yPred = model.predict(xTest_ohe)
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred))

197/197 [==============================] - 0s 1ms/sample
              precision    recall  f1-score   support

           0       0.86      0.95      0.90        99
           1       0.94      0.85      0.89        98

    accuracy                           0.90       197
   macro avg       0.90      0.90      0.90       197
weighted avg       0.90      0.90      0.90       197



In [12]:
import graphviz

In [217]:
n_values=5

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from tensorflow.keras.utils import to_categorical



ohenc = OneHotEncoder(sparse=False, categories="auto",dtype=int)
ohenc.fit(np.concatenate((xTrain_pad, xTest_pad)))

enc = ohenc.transform(xTrain_pad)
#print(list(xTrain_pad[0]))

samp = np.array([[0,71,71,71,84],[0,65,71,71,84]])
print(samp.shape, samp[0], list(set(samp[0])))
print(samp.T.shape, list(samp.T))

base_encoder = LabelEncoder()
base_encoder.fit(samp.flatten())
samp_num = np.array([base_encoder.transform(s) for s in samp])
samp_num

#ord_encoder =  OrdinalEncoder(dtype=int)
#samp_ord = ord_encoder.fit_transform(samp.T)
#print(samp_ord)
#samp2 = np.array([[0,1,0,0,0],[0,0,0,0,0]])
#ord_encoder.inverse_transform(samp2)

#samp.reshape(-1, 1)

onehotencoder = OneHotEncoder(categories='auto', dtype=int, sparse=False)

samp_num1 = samp_num[0].reshape(len(samp_num[0]), 1)
samp_oh = onehotencoder.fit_transform(samp_num1)
samp_oh
#enc = ohenc.transform(samp)

#print(list(enc[0]))
#enc_samp
#to_categorical(samp[0], num_classes=5)

(2, 5) [ 0 71 71 71 84] [0, 84, 71]
(5, 2) [array([0, 0]), array([71, 65]), array([71, 71]), array([71, 71]), array([84, 84])]


array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [230]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from tensorflow.keras.utils import to_categorical
print(xTrain_pad.flatten())
print(np.concatenate((xTrain_pad, xTest_pad), axis=None))

[ 0  0  0 ... 67 65 67]
[ 0  0  0 ... 71 84 65]


In [334]:
base_encoder = LabelEncoder()
base_encoder.fit(np.concatenate((xTrain_pad, xTest_pad), axis=None))

xTrain_num = np.array([base_encoder.transform(seq) for seq in xTrain_pad])
xTest_num = np.array([base_encoder.transform(seq) for seq in xTest_pad])
xTrain_num

array([[0, 0, 0, ..., 5, 5, 5],
       [0, 0, 0, ..., 5, 2, 2],
       [0, 0, 0, ..., 5, 2, 1],
       ...,
       [0, 0, 0, ..., 5, 3, 3],
       [0, 0, 0, ..., 3, 1, 5],
       [0, 0, 0, ..., 2, 1, 2]])

In [328]:
#xTrain_prep = np.array([seq.reshape(len(seq), 1) for seq in xTrain_num])
#print(xTrain_prep.shape)
print(xTrain_num.shape)
print(list(xTrain_num[0]))
print(xTrain_num[0].reshape(1, -1) )
onehotencoder = OneHotEncoder(categories='auto', dtype=int, sparse=False)
onehotencoder.fit(xTrain_num[0].reshape(1, -1))

#xTrain_ohe = [onehotencoder.transform(seq) for seq in xTrain_num]

#print(list(xTrain_ohe[0]))
onehotencoder.transform(xTrain_num[0].reshape(1, -1)).shape

(787, 1004)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

(1, 1004)

In [317]:
print(np.array(xTrain_ohe).shape)

#xTrain_ohe = np.array(xTrain_ohe)
#xTrain_ohe

(787, 1, 3346)


In [296]:
print(xTrain_ohe.shape)
lens=set()
for x in xTrain_ohe:
    lens.add(len(x))
lens    

(787,)


{4016, 5020, 6024}

In [363]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils

def create_model_gridsearch_ohe(dropout=0.5):
    # create the model
    model = keras.Sequential()
    model.add(keras.layers.Embedding(6, 3, input_length=1004 ) )
    model.add(keras.layers.Conv1D(16, 3, activation='relu', padding='same'))
    model.add(keras.layers.Conv1D(16, 6, activation='relu', padding='same'))
    #model.add(keras.layers.Conv1D(8, 3, activation='relu', padding='same', input_shape=(3365, 1)))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(64, 16, activation='relu'))
    model.add(keras.layers.Conv1D(64, 32, activation='relu'))
    #model.add(keras.layers.MaxPooling1D(2))
    #model.add(keras.layers.Conv1D(128, 16, activation='relu'))
    #model.add(keras.layers.Conv1D(32, 32, activation='relu'))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(50, activation='relu'))
    #model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    optAdam = keras.optimizers.Adam()
    model.compile(optimizer=optAdam, loss='binary_crossentropy', metrics=['accuracy'])
    
    print(model.summary())
    
    return model


In [348]:
import pydot
import graphviz

ModuleNotFoundError: No module named 'graphviz'

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_55 (Embedding)     (None, 1004, 3)           18        
_________________________________________________________________
conv1d_164 (Conv1D)          (None, 1004, 16)          160       
_________________________________________________________________
conv1d_165 (Conv1D)          (None, 1004, 16)          1552      
_________________________________________________________________
max_pooling1d_127 (MaxPoolin (None, 502, 16)           0         
_________________________________________________________________
conv1d_166 (Conv1D)          (None, 487, 64)           16448     
_________________________________________________________________
conv1d_167 (Conv1D)          (None, 456, 64)           131136    
_________________________________________________________________
global_average_pooling1d_13  (None, 64)                0         
__________

In [335]:
yPred = model.predict(xTest_num)
#yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred))


197/197 [==============================] - 1s 3ms/sample
              precision    recall  f1-score   support

           0       0.87      0.90      0.89        99
           1       0.89      0.87      0.88        98

    accuracy                           0.88       197
   macro avg       0.88      0.88      0.88       197
weighted avg       0.88      0.88      0.88       197



In [123]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model_embedding, verbose=1)
# define the grid search parameters
batch_size = [32]
epochs = [5]
emb_in=[3,6,9] #6
emb_out=[2,3,4] #3
conv1_ks=[8,16,32] #16
conv1_w=[3,5]
conv2_ks=[8,16,32]
conv2_w=[7,9]
dense=[5,10,20,40]

param_grid = dict(batch_size=batch_size,
                 epochs=epochs,
                 emb_in=emb_in,
                 emb_out=emb_out,
                 conv1_ks=conv1_ks,
                 conv1_w=conv1_w,
                 conv2_ks=conv2_ks,
                 conv2_w=conv2_w,
                 dense=dense)



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=1)

grid_result = grid.fit(xTrain_oh, yTrain_enc)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 2 folds for each of 1296 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/wehrmann/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 25.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 51.8min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 82.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 129.1min
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed: 141.0min finished
/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The defau

Epoch 1/5
787/787 [==============================] - 2s 3ms/sample - loss: 0.6987 - acc: 0.4828
Epoch 2/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.6830 - acc: 0.7001
Epoch 3/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.5211 - acc: 0.8463
Epoch 4/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2746 - acc: 0.9098
Epoch 5/5
787/787 [==============================] - 2s 2ms/sample - loss: 0.2315 - acc: 0.9250
Best: 0.921220 using {'batch_size': 32, 'conv1_ks': 16, 'conv1_w': 5, 'conv2_ks': 32, 'conv2_w': 9, 'dense': 40, 'emb_in': 6, 'emb_out': 3, 'epochs': 5}
0.739517 (0.136290) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 2, 'epochs': 5}
0.677255 (0.211338) with: {'batch_size': 32, 'conv1_ks': 8, 'conv1_w': 3, 'conv2_ks': 8, 'conv2_w': 7, 'dense': 5, 'emb_in': 3, 'emb_out': 3, 'epochs': 5}
0.673443 (0.207531) with: {'batch_size': 32, 'conv1_ks': 

In [124]:
grid_result.best_params_

{'batch_size': 32,
 'conv1_ks': 16,
 'conv1_w': 5,
 'conv2_ks': 32,
 'conv2_w': 9,
 'dense': 40,
 'emb_in': 6,
 'emb_out': 3,
 'epochs': 5}

In [125]:
model = grid_result.best_estimator_

In [126]:
yPred = model.predict(ohenc.transform(xTest_pad))
yPred_class = np.array([np.argmax(row) for row in yPred])

from sklearn.metrics import classification_report

print(classification_report( encoder.transform(yTest.Class.to_list()), yPred_class))

197/197 [==============================] - 0s 2ms/sample
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        99
           1       0.00      0.00      0.00        98

    accuracy                           0.50       197
   macro avg       0.25      0.50      0.33       197
weighted avg       0.25      0.50      0.34       197



/home/wehrmann/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
